# Week 17: Options Pricing & Deep Hedging Theory

**Core Topics**: Black-Scholes Model, Greeks, Deep Hedging with Neural Networks

---

## 1. Black-Scholes Model

### 1.1 Assumptions
- Stock price follows Geometric Brownian Motion (GBM)
- No dividends during option life
- No arbitrage opportunities
- Risk-free rate and volatility are constant
- Continuous trading with no transaction costs
- European-style options only

### 1.2 Stock Price Dynamics (GBM)

$$dS_t = \mu S_t dt + \sigma S_t dW_t$$

Where:
- $S_t$ = Stock price at time $t$
- $\mu$ = Drift (expected return)
- $\sigma$ = Volatility
- $W_t$ = Wiener process (Brownian motion)

### 1.3 Black-Scholes PDE

$$\frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + rS\frac{\partial V}{\partial S} - rV = 0$$

### 1.4 Black-Scholes Formulas

**Call Option:**
$$C = S_0 N(d_1) - Ke^{-rT}N(d_2)$$

**Put Option:**
$$P = Ke^{-rT}N(-d_2) - S_0 N(-d_1)$$

Where:
$$d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$$
$$d_2 = d_1 - \sigma\sqrt{T}$$

- $S_0$ = Current stock price
- $K$ = Strike price
- $r$ = Risk-free rate
- $T$ = Time to expiration
- $N(\cdot)$ = Standard normal CDF

In [ ]:
import numpy as np
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type='call'):
    """
    Black-Scholes option pricing formula.
    
    Parameters:
    -----------
    S : float - Current stock price
    K : float - Strike price
    T : float - Time to expiration (years)
    r : float - Risk-free rate
    sigma : float - Volatility
    option_type : str - 'call' or 'put'
    
    Returns:
    --------
    float : Option price
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:  # put
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return price

# Example
S, K, T, r, sigma = 100, 100, 1.0, 0.05, 0.2
print(f"Call Price: ${black_scholes(S, K, T, r, sigma, 'call'):.2f}")
print(f"Put Price: ${black_scholes(S, K, T, r, sigma, 'put'):.2f}")

---

## 2. The Greeks

Greeks measure sensitivity of option price to various parameters.

| Greek | Symbol | Measures | Formula (Call) |
|-------|--------|----------|----------------|
| **Delta** | $\Delta$ | Price sensitivity to $S$ | $N(d_1)$ |
| **Gamma** | $\Gamma$ | Delta sensitivity to $S$ | $\frac{N'(d_1)}{S\sigma\sqrt{T}}$ |
| **Theta** | $\Theta$ | Time decay | $-\frac{S N'(d_1)\sigma}{2\sqrt{T}} - rKe^{-rT}N(d_2)$ |
| **Vega** | $\mathcal{V}$ | Volatility sensitivity | $S\sqrt{T}N'(d_1)$ |
| **Rho** | $\rho$ | Interest rate sensitivity | $KTe^{-rT}N(d_2)$ |

### 2.1 Key Relationships

- **Delta-Gamma Relationship**: $\Gamma = \frac{\partial \Delta}{\partial S}$
- **Put-Call Delta**: $\Delta_{put} = \Delta_{call} - 1$
- **Gamma** is same for puts and calls with same strike/expiry
- **Vega** is same for puts and calls (same strike/expiry)

In [ ]:
def greeks(S, K, T, r, sigma, option_type='call'):
    """
    Calculate all Greeks for a European option.
    
    Returns:
    --------
    dict : Dictionary containing all Greeks
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Common terms
    n_d1 = norm.pdf(d1)  # N'(d1)
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    
    # Delta
    if option_type == 'call':
        delta = N_d1
    else:
        delta = N_d1 - 1
    
    # Gamma (same for call and put)
    gamma = n_d1 / (S * sigma * np.sqrt(T))
    
    # Theta
    if option_type == 'call':
        theta = (-S * n_d1 * sigma / (2 * np.sqrt(T)) 
                 - r * K * np.exp(-r * T) * N_d2)
    else:
        theta = (-S * n_d1 * sigma / (2 * np.sqrt(T)) 
                 + r * K * np.exp(-r * T) * norm.cdf(-d2))
    
    # Vega (same for call and put)
    vega = S * np.sqrt(T) * n_d1
    
    # Rho
    if option_type == 'call':
        rho = K * T * np.exp(-r * T) * N_d2
    else:
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    return {
        'delta': delta,
        'gamma': gamma,
        'theta': theta / 365,  # Daily theta
        'vega': vega / 100,    # Per 1% vol change
        'rho': rho / 100       # Per 1% rate change
    }

# Example
g = greeks(100, 100, 1.0, 0.05, 0.2, 'call')
print("Greeks for ATM Call Option:")
for name, value in g.items():
    print(f"  {name.capitalize()}: {value:.4f}")

---

## 3. Traditional Delta Hedging

### 3.1 Concept

Delta hedging aims to create a **delta-neutral portfolio**:

$$\Pi = V - \Delta \cdot S$$

Where:
- $\Pi$ = Hedged portfolio value
- $V$ = Option value
- $\Delta$ = Option delta
- $S$ = Stock position

### 3.2 Limitations of Delta Hedging

1. **Discrete Rebalancing**: Real-world hedging can't be continuous
2. **Transaction Costs**: Frequent rebalancing is expensive
3. **Model Risk**: Greeks assume Black-Scholes holds
4. **Volatility Uncertainty**: $\sigma$ is not known precisely
5. **Gamma Risk**: Delta changes rapidly near expiry for ATM options

In [ ]:
def simulate_delta_hedge(S0, K, T, r, sigma, n_steps, n_paths, transaction_cost=0.001):
    """
    Simulate delta hedging with transaction costs.
    
    Returns:
    --------
    dict : Hedging P&L statistics
    """
    dt = T / n_steps
    
    # Simulate GBM paths
    np.random.seed(42)
    Z = np.random.standard_normal((n_paths, n_steps))
    S = np.zeros((n_paths, n_steps + 1))
    S[:, 0] = S0
    
    for t in range(n_steps):
        S[:, t+1] = S[:, t] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z[:, t])
    
    # Initial option price and delta
    initial_price = black_scholes(S0, K, T, r, sigma, 'call')
    
    # Track hedge P&L
    pnl = np.zeros(n_paths)
    
    for path in range(n_paths):
        cash = initial_price  # Received premium
        stock_held = 0
        total_tc = 0
        
        for t in range(n_steps):
            time_remaining = T - t * dt
            if time_remaining > 1e-6:
                # Calculate delta
                d1 = (np.log(S[path, t]/K) + (r + 0.5*sigma**2)*time_remaining) / (sigma*np.sqrt(time_remaining))
                delta = norm.cdf(d1)
            else:
                delta = 1.0 if S[path, t] > K else 0.0
            
            # Rebalance
            trade = delta - stock_held
            tc = abs(trade) * S[path, t] * transaction_cost
            cash -= trade * S[path, t] + tc
            total_tc += tc
            stock_held = delta
            
            # Accrue interest
            cash *= np.exp(r * dt)
        
        # Final payoff
        payoff = max(S[path, -1] - K, 0)
        final_value = cash + stock_held * S[path, -1] - payoff
        pnl[path] = final_value
    
    return {
        'mean_pnl': np.mean(pnl),
        'std_pnl': np.std(pnl),
        'min_pnl': np.min(pnl),
        'max_pnl': np.max(pnl)
    }

# Simulate
results = simulate_delta_hedge(100, 100, 1.0, 0.05, 0.2, n_steps=52, n_paths=1000)
print("Delta Hedging Results (weekly rebalancing):")
for k, v in results.items():
    print(f"  {k}: ${v:.4f}")

---

## 4. Deep Hedging

### 4.1 Motivation

**Why Deep Hedging?**
- Traditional hedging assumes idealized market conditions
- Real markets have: transaction costs, discrete hedging, liquidity constraints
- Deep learning can learn **optimal hedging strategies** that account for these frictions

### 4.2 Problem Formulation

**Objective**: Find hedging strategy $\delta_t$ that minimizes a risk measure of the P&L:

$$\min_{\delta} \rho\left(-Z + \sum_{t=0}^{T-1} \delta_t (S_{t+1} - S_t) - C_t(\delta_t - \delta_{t-1})\right)$$

Where:
- $Z$ = Option payoff at maturity
- $\delta_t$ = Hedge position at time $t$ (learned by NN)
- $C_t$ = Transaction costs
- $\rho$ = Risk measure (e.g., CVaR, variance)

### 4.3 Architecture

```
Input Features → Neural Network → Hedge Ratio δ
     ↓                                    ↓
[S_t, t, σ, ...]  →  [Dense layers]  →  δ_t ∈ [0, 1]
```

**Input features typically include:**
- Current stock price $S_t$ (or log-moneyness)
- Time to expiry $\tau$
- Current hedge position $\delta_{t-1}$
- Implied/realized volatility
- Previous price changes

### 4.4 Loss Functions for Deep Hedging

**1. Mean-Variance (Quadratic Penalty)**
$$\mathcal{L} = \mathbb{E}[PnL^2] + \lambda \cdot \text{Var}(PnL)$$

**2. CVaR (Conditional Value at Risk)**
$$\text{CVaR}_\alpha = \mathbb{E}[PnL | PnL \leq \text{VaR}_\alpha]$$

**3. Entropic Risk Measure**
$$\rho_\gamma(X) = \frac{1}{\gamma}\log\mathbb{E}[e^{-\gamma X}]$$

Where $\gamma$ controls risk aversion (higher = more risk-averse)

In [ ]:
import torch
import torch.nn as nn

class DeepHedgingNetwork(nn.Module):
    """
    Neural network for learning optimal hedging strategy.
    
    Architecture:
    - Input: [log_moneyness, time_to_expiry, prev_delta]
    - Output: hedge ratio δ ∈ [0, 1]
    """
    def __init__(self, input_dim=3, hidden_dims=[64, 64, 32]):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_dim)
            ])
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, 1))
        layers.append(nn.Sigmoid())  # Output in [0, 1]
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)


def entropic_risk_measure(pnl, gamma=1.0):
    """
    Entropic risk measure: ρ(X) = (1/γ) * log(E[exp(-γX)])
    
    Higher gamma = more risk averse
    """
    return (1.0 / gamma) * torch.logsumexp(-gamma * pnl, dim=0) - np.log(len(pnl)) / gamma


def cvar_loss(pnl, alpha=0.05):
    """
    CVaR (Expected Shortfall) at level alpha.
    Returns the mean of the worst alpha% of outcomes.
    """
    sorted_pnl, _ = torch.sort(pnl)
    n_tail = int(len(pnl) * alpha)
    return -sorted_pnl[:n_tail].mean()  # Negative because we minimize

print("Deep Hedging Network Architecture:")
model = DeepHedgingNetwork()
print(model)

### 4.5 Training Loop (Conceptual)

```python
for epoch in range(n_epochs):
    # 1. Simulate price paths
    S = simulate_gbm_paths(S0, r, sigma, T, n_steps, batch_size)
    
    # 2. Forward pass: compute hedges at each time step
    delta_prev = torch.zeros(batch_size)
    hedge_pnl = 0
    transaction_costs = 0
    
    for t in range(n_steps):
        # Features: log-moneyness, time-to-expiry, previous delta
        features = [log(S[:,t]/K), (T-t*dt)/T, delta_prev]
        
        # Neural network outputs hedge ratio
        delta_t = model(features)
        
        # Accumulate hedge P&L
        hedge_pnl += delta_t * (S[:,t+1] - S[:,t])
        
        # Transaction costs
        transaction_costs += cost_rate * |delta_t - delta_prev| * S[:,t]
        
        delta_prev = delta_t
    
    # 3. Compute total P&L (option premium + hedge - payoff - costs)
    option_payoff = max(S[:,-1] - K, 0)
    total_pnl = premium + hedge_pnl - option_payoff - transaction_costs
    
    # 4. Compute risk measure and backprop
    loss = risk_measure(total_pnl)  # e.g., CVaR or entropic
    loss.backward()
    optimizer.step()
```

In [ ]:
class DeepHedgingSimulator:
    """
    End-to-end deep hedging training framework.
    """
    def __init__(self, S0, K, T, r, sigma, n_steps, transaction_cost=0.001):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.n_steps = n_steps
        self.dt = T / n_steps
        self.tc = transaction_cost
        
        # Black-Scholes price as initial premium
        self.premium = black_scholes(S0, K, T, r, sigma, 'call')
        
        self.model = DeepHedgingNetwork(input_dim=3)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
    
    def simulate_paths(self, batch_size):
        """Simulate GBM paths using PyTorch."""
        Z = torch.randn(batch_size, self.n_steps)
        S = torch.zeros(batch_size, self.n_steps + 1)
        S[:, 0] = self.S0
        
        for t in range(self.n_steps):
            S[:, t+1] = S[:, t] * torch.exp(
                (self.r - 0.5*self.sigma**2)*self.dt + 
                self.sigma*np.sqrt(self.dt)*Z[:, t]
            )
        return S
    
    def compute_pnl(self, S):
        """Compute hedging P&L for a batch of paths."""
        batch_size = S.shape[0]
        delta_prev = torch.zeros(batch_size)
        hedge_pnl = torch.zeros(batch_size)
        total_tc = torch.zeros(batch_size)
        
        self.model.train()
        
        for t in range(self.n_steps):
            tau = (self.T - t * self.dt) / self.T  # Normalized time to expiry
            log_moneyness = torch.log(S[:, t] / self.K)
            
            # Features: [log_moneyness, time_to_expiry, prev_delta]
            features = torch.stack([log_moneyness, 
                                   torch.full((batch_size,), tau),
                                   delta_prev], dim=1)
            
            # Get hedge ratio from neural network
            delta_t = self.model(features).squeeze()
            
            # Hedge P&L
            dS = S[:, t+1] - S[:, t]
            hedge_pnl += delta_t * dS
            
            # Transaction costs
            total_tc += self.tc * torch.abs(delta_t - delta_prev) * S[:, t]
            
            delta_prev = delta_t.detach()
        
        # Option payoff
        payoff = torch.relu(S[:, -1] - self.K)
        
        # Total P&L: premium received + hedge gains - payoff - costs
        pnl = self.premium + hedge_pnl - payoff - total_tc
        
        return pnl
    
    def train_step(self, batch_size=256, risk_measure='entropic', gamma=1.0):
        """Single training step."""
        self.optimizer.zero_grad()
        
        S = self.simulate_paths(batch_size)
        pnl = self.compute_pnl(S)
        
        if risk_measure == 'entropic':
            loss = entropic_risk_measure(pnl, gamma)
        elif risk_measure == 'cvar':
            loss = cvar_loss(pnl, alpha=0.05)
        else:  # variance
            loss = pnl.var() - pnl.mean()
        
        loss.backward()
        self.optimizer.step()
        
        return loss.item(), pnl.mean().item(), pnl.std().item()

print("Deep Hedging Simulator initialized")
print(f"Black-Scholes premium: ${black_scholes(100, 100, 1.0, 0.05, 0.2):.2f}")

---

## 5. Key Insights: Deep Hedging vs Traditional

| Aspect | Delta Hedging | Deep Hedging |
|--------|--------------|---------------|
| **Transaction Costs** | Ignored or approximated | Explicitly optimized |
| **Rebalancing** | Fixed frequency | Learned (can be state-dependent) |
| **Model Assumptions** | Requires BS assumptions | Model-free (learns from data) |
| **Risk Measure** | Variance implicit | Flexible (CVaR, entropic, etc.) |
| **Computation** | Analytical | Requires training |
| **Interpretability** | Clear (delta = sensitivity) | Black box |

### 5.1 When to Use Deep Hedging

✅ **Ideal for:**
- High transaction cost environments
- Exotic derivatives with no closed-form Greeks
- When model risk is high (vol smile, jumps)
- Portfolio hedging across multiple instruments

❌ **Less suitable for:**
- Simple vanilla options in liquid markets
- When interpretability is critical
- Limited computational resources

---

## 6. Interview Quick Reference

### Q1: Derive Black-Scholes PDE
- Start with GBM: $dS = \mu S dt + \sigma S dW$
- Apply Itô's lemma to option $V(S,t)$
- Form riskless portfolio: $\Pi = V - \Delta S$
- Set $d\Pi = r\Pi dt$ (no arbitrage)

### Q2: Why is Gamma highest for ATM options near expiry?
- ATM options have delta ~0.5, which changes rapidly with small $S$ moves
- Near expiry, option must decide between 0 (OTM) or 1 (ITM)
- Formula: $\Gamma = \frac{N'(d_1)}{S\sigma\sqrt{T}}$ → explodes as $T \to 0$ for ATM

### Q3: What is deep hedging?
- Uses neural networks to learn optimal hedging strategies
- Minimizes a risk measure (CVaR, entropic) of hedging P&L
- Accounts for transaction costs and market frictions
- Model-free: doesn't assume Black-Scholes dynamics

### Q4: Why use entropic risk measure?
- Convex and coherent risk measure
- $\gamma$ parameter controls risk aversion
- Differentiable → compatible with gradient descent
- As $\gamma \to 0$: approaches expected value (risk neutral)
- As $\gamma \to \infty$: approaches worst-case (max loss)

---

## 7. References

1. **Black, F. & Scholes, M. (1973)** - "The Pricing of Options and Corporate Liabilities"
2. **Buehler, H. et al. (2019)** - "Deep Hedging" (Quantitative Finance)
3. **Hull, J.** - "Options, Futures, and Other Derivatives" (Chapter on Greeks)
4. **Ruf, J. & Wang, W. (2020)** - "Neural Networks for Option Pricing and Hedging"